In [1975]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
from category_encoders import OneHotEncoder,BinaryEncoder
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split
import re
from scipy.stats import pointbiserialr
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestRegressor
import missingno as msno
from sklearn.preprocessing import RobustScaler
import matplotlib.patches as mpatches
import math
from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [1976]:
df1=pd.read_csv("info_base_games.csv")
df1.head()

,appid,name,metacritic,steam_achievements,steam_trading_cards,workshop_support,genres,achievements_total,release_date,supported_platforms
0,2574000,Femboy Burgers,NaN,True,True,True,"Casual, Indie",NaN,"Oct 9, 2023","['windows', 'mac', 'linux']"
1,2574120,PPA Pickleball Tour 2025,NaN,True,True,True,"Indie, Simulation, Sports",18,"Jul 16, 2024","['windows', 'mac', 'linux']"
2,2573200,Squeaky Squad,NaN,True,True,True,"Action, Adventure, Indie",27,"Mar 29, 2024","['windows', 'mac', 'linux']"
3,2573440,Paradox Metal,NaN,True,True,True,"Action, Early Access",NaN,Coming soon,"['windows', 'mac', 'linux']"
4,2569520,Naturpark Lillebælt VR,NaN,True,True,True,"Action, Adventure",NaN,"Sep 18, 2023","['windows', 'mac', 'linux']"


In [1977]:
def data_info(data):

    """
    This function returns a DataFrame containing the summary information for each column 
    """

    Names=[col for col in data]
    data_types=[data[col].dtype for col in data.columns]
    top_10_unique_values=[data[col].value_counts().head(10).index.to_list() for col in data.columns]
    nunique_values=[data[col].nunique() for col in data.columns]
    nulls=[data[col].isnull().sum() for col in data.columns]
    percent_of_Nulls= [data[col].isnull().sum()/len(data)*100 for col in data.columns]
    duplicates=data.duplicated().sum()


    info_df=pd.DataFrame({'Name':Names,
                          'Data_Type':data_types,
                          'Top_10_Unique_Values':top_10_unique_values,
                          'Nunique_Values':nunique_values,
                          'Nulls':nulls,
                          'Percent_of_Nulls':percent_of_Nulls,
                          'Duplicates':duplicates})
    return info_df

In [1978]:
df1.shape

(99167, 10)

In [1979]:
data_info(df1)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,appid,object,"[1811260, 2574000, 1024400, 1023570, 1023560, ...",99166,0,0.000000,0
1,name,object,"[Echoes, Delirium, Dark Light, Creator-initial...",95004,18,0.018151,0
2,metacritic,object,"[80, 76, 75, 68, 73, 81, 79.0, 81.0, 80.0, 77]",170,96148,96.955640,0
3,steam_achievements,bool,"[False, True]",2,0,0.000000,0
4,steam_trading_cards,bool,"[False, True]",2,0,0.000000,0
5,workshop_support,bool,"[False, True]",2,0,0.000000,0
6,genres,object,"[Action, Adventure, Indie, Casual, Indie, Acti...",2699,4778,4.818135,0
7,achievements_total,object,"[10, 12, 6, 20, 15, 10.0, 5, 8, 11, 16]",695,61052,61.564835,0
8,release_date,object,"[Coming soon, To be announced, 2025, Q1 2025, ...",4788,306,0.308570,0
9,supported_platforms,object,"[['windows'], ['windows', 'mac', 'linux'], ['w...",7,0,0.000000,0


In [1980]:
df2=pd.read_csv("demos.csv")
df2.head()

,Unnamed: 0,full_game_appid,demo_appid,name
0,0,2214650,2573370,Rolando Deluxe Demo
1,1,1439980,2573460,Outrunner: Neon Nights Demo
2,2,2412240,2572840,Bubble Ghost Remake Demo
3,3,2448830,2572240,Time Handlers Demo
4,4,2379590,2570800,Hope's Final Defense Demo


In [1981]:
df2.shape

(15449, 4)

In [1982]:
data_info(df2)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,Unnamed: 0,int64,"[0, 10264, 10296, 10297, 10298, 10299, 10300, ...",15449,0,0.000000,0
1,full_game_appid,object,"[2214650, 1646810, 1831020, 1832380, 1146420, ...",15449,0,0.000000,0
2,demo_appid,object,"[2573370, 1855960, 1846470, 1845950, 1845990, ...",15449,0,0.000000,0
3,name,object,"[Exodus Demo, #NAME?, Automatica Demo, Dungeon...",15425,1,0.006473,0


In [1983]:
df3=pd.read_csv("dlcs.csv")
df3.head()

,base_appid,dlc_appid,name
0,1786750,2568660,家出王女 - 全年齢版ストーリー&グラフィック追加 DLC
1,1981700,2563730,Jacob's Quest - Voyage
2,2009450,2552980,Invector: Rhythm Galaxy - Latin Power Song Pack
3,1133420,2550750,Hero or Villain: Genesis — Supercharged!
4,2533950,2551000,Hot And Lovely ：Uniform - adult patch


In [1984]:
df3.shape

(5445, 3)

In [1985]:
data_info(df3)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,base_appid,object,"[1786750, 638850, 883390, 880550, 771500, 8131...",5445,0,0.0,0
1,dlc_appid,object,"[2568660, 890020, 888110, 887980, 888590, 8890...",5445,0,0.0,0
2,name,object,"[Dungeon Full Dive - True Supporter Dice, Doll...",5443,0,0.0,0


In [1986]:
df4=pd.read_csv("gamalytic_steam_games.csv")
df4.head()

,steamId,price,copiesSold,publisherClass,reviewScore,aiContent
0,730,0.0,302158048,AAA,87,NaN
1,570,0.0,212896574,AAA,82,NaN
2,578080,0.0,161971233,AAA,59,NaN
3,440,0.0,99060457,AAA,90,NaN
4,1172470,0.0,67554185,AAA,67,NaN


In [1987]:
df4.shape

(93338, 6)

In [1988]:
data_info(df4)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,steamId,int64,"[730, 848890, 3461160, 770850, 2304650, 109873...",93338,0,0.0,0
1,price,float64,"[0.0, 4.99, 9.99, 0.99, 1.99, 2.99, 3.99, 14.9...",350,0,0.0,0
2,copiesSold,int64,"[1, 15, 30, 36, 45, 75, 72, 18, 60, 90]",22474,0,0.0,0
3,publisherClass,object,"[Hobbyist, Indie, AA, AAA]",4,0,0.0,0
4,reviewScore,int64,"[100, 0, 50, 67, 75, 83, 80, 88, 86, 89]",99,0,0.0,0
5,aiContent,float64,[],0,93338,100.0,0


# Convert Appid Data Type in all Data Frames

In [1989]:
df1["appid"] = pd.to_numeric(df1['appid'], errors='coerce').astype('Int64')
df2["appid"] = pd.to_numeric(df2['full_game_appid'], errors='coerce').astype('Int64')
df3["appid"] = pd.to_numeric(df3['base_appid'], errors='coerce').astype('Int64')
df4["appid"] = pd.to_numeric(df4['steamId'], errors='coerce').astype('Int64')

# Merge

In [1990]:
merged = pd.merge(df1,   df4, on='appid', how='inner')
merged = pd.merge(merged, df2,   on='appid', how='left')
df = pd.merge(merged, df3,  on='appid', how='left')

In [1991]:
data_info(df)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,appid,Int64,"[3297920, 3175890, 2317310, 2316930, 3332170, ...",67909,0,0.000000,0
1,name_x,object,"[Echoes, Delirium, Alone, Zombie Hunter, Lost,...",67531,0,0.000000,0
2,metacritic,object,"[80, 76, 73, 68, 75, 81, 79.0, 81.0, 80.0, 77]",169,66495,95.775480,0
3,steam_achievements,bool,"[True, False]",2,0,0.000000,0
4,steam_trading_cards,bool,"[False, True]",2,0,0.000000,0
5,workshop_support,bool,"[False, True]",2,0,0.000000,0
6,genres,object,"[Casual, Indie, Action, Indie, Action, Adventu...",2016,104,0.149795,0
7,achievements_total,object,"[10, 12, 6, 20, 15, 10.0, 8, 5, 11, 16]",691,32133,46.282480,0
8,release_date,object,"[Coming soon, Q1 2025, Oct 31, 2024, Dec 5, 20...",4541,2,0.002881,0
9,supported_platforms,object,"[['windows'], ['windows', 'mac', 'linux'], ['w...",7,0,0.000000,0


# Dropping Columns With High Null%

In [1992]:
dropped_list=["metacritic","achievements_total","aiContent","Unnamed: 0","full_game_appid","demo_appid","name_y",
              "base_appid","dlc_appid","name"]

df.drop(columns=dropped_list,inplace=True)

In [1993]:
data_info(df)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,appid,Int64,"[3297920, 3175890, 2317310, 2316930, 3332170, ...",67909,0,0.000000,2
1,name_x,object,"[Echoes, Delirium, Alone, Zombie Hunter, Lost,...",67531,0,0.000000,2
2,steam_achievements,bool,"[True, False]",2,0,0.000000,2
3,steam_trading_cards,bool,"[False, True]",2,0,0.000000,2
4,workshop_support,bool,"[False, True]",2,0,0.000000,2
5,genres,object,"[Casual, Indie, Action, Indie, Action, Adventu...",2016,104,0.149795,2
6,release_date,object,"[Coming soon, Q1 2025, Oct 31, 2024, Dec 5, 20...",4541,2,0.002881,2
7,supported_platforms,object,"[['windows'], ['windows', 'mac', 'linux'], ['w...",7,0,0.000000,2
8,steamId,int64,"[3297920, 3175890, 2317310, 2316930, 3332170, ...",67909,0,0.000000,2
9,price,float64,"[0.0, 4.99, 0.99, 9.99, 1.99, 2.99, 3.99, 14.9...",295,0,0.000000,2


# Measure Correlation with id's

In [1994]:
list1=["appid","steamId","copiesSold"]
cor=df[list1].corr()
cor

,appid,steamId,copiesSold
appid,1.000000,1.000000,-0.053396
steamId,1.000000,1.000000,-0.053396
copiesSold,-0.053396,-0.053396,1.000000


# Dropping columns with high unique values and irrelevant to target

In [1995]:
df.drop(["steamId","appid"],axis=1,inplace=True)

In [1996]:
data_info(df)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,name_x,object,"[Echoes, Delirium, Alone, Zombie Hunter, Lost,...",67531,0,0.000000,2
1,steam_achievements,bool,"[True, False]",2,0,0.000000,2
2,steam_trading_cards,bool,"[False, True]",2,0,0.000000,2
3,workshop_support,bool,"[False, True]",2,0,0.000000,2
4,genres,object,"[Casual, Indie, Action, Indie, Action, Adventu...",2016,104,0.149795,2
5,release_date,object,"[Coming soon, Q1 2025, Oct 31, 2024, Dec 5, 20...",4541,2,0.002881,2
6,supported_platforms,object,"[['windows'], ['windows', 'mac', 'linux'], ['w...",7,0,0.000000,2
7,price,float64,"[0.0, 4.99, 0.99, 9.99, 1.99, 2.99, 3.99, 14.9...",295,0,0.000000,2
8,copiesSold,int64,"[1, 15, 30, 36, 45, 18, 75, 72, 60, 90]",17774,0,0.000000,2
9,publisherClass,object,"[Hobbyist, Indie, AA, AAA]",4,0,0.000000,2


# Preprocess on all merged Data (Handle Nulls, Duplicates , Outliers)

Missing Values are in Genres & release_date column

In [1997]:
df["genres"]=df["genres"].fillna(method='ffill').fillna(method='bfill')
df.dropna(subset=["release_date"], inplace=True)

In [1998]:
data_info(df)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,name_x,object,"[Delirium, Echoes, Arena, Dodge, Zombie Hunter...",67529,0,0.0,2
1,steam_achievements,bool,"[True, False]",2,0,0.0,2
2,steam_trading_cards,bool,"[False, True]",2,0,0.0,2
3,workshop_support,bool,"[False, True]",2,0,0.0,2
4,genres,object,"[Casual, Indie, Action, Indie, Action, Adventu...",2016,0,0.0,2
5,release_date,object,"[Coming soon, Q1 2025, Oct 31, 2024, Dec 5, 20...",4541,0,0.0,2
6,supported_platforms,object,"[['windows'], ['windows', 'mac', 'linux'], ['w...",7,0,0.0,2
7,price,float64,"[0.0, 4.99, 0.99, 9.99, 1.99, 2.99, 3.99, 14.9...",295,0,0.0,2
8,copiesSold,int64,"[1, 15, 30, 36, 45, 18, 75, 72, 60, 90]",17772,0,0.0,2
9,publisherClass,object,"[Hobbyist, Indie, AA, AAA]",4,0,0.0,2


In [1999]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2000]:
year_pattern = r"\b(\d{4})\b"
df["release_date"] = [re.search(year_pattern, date).group(1) if re.search(year_pattern, date) else date for date in df["release_date"]]
df["release_date"] = df["release_date"].str.replace(r".*-25$", "2025", regex=True)
df["release_date"]=df["release_date"].apply(lambda x: '2026' if x in ["Coming soon", "To be announced"] else x)

In [2001]:
data_info(df)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,name_x,object,"[Delirium, Echoes, Arena, Dodge, Zombie Hunter...",67529,0,0.0,2
1,steam_achievements,bool,"[True, False]",2,0,0.0,2
2,steam_trading_cards,bool,"[False, True]",2,0,0.0,2
3,workshop_support,bool,"[False, True]",2,0,0.0,2
4,genres,object,"[Casual, Indie, Action, Indie, Action, Adventu...",2016,0,0.0,2
5,release_date,object,"[2024, 2021, 2023, 2018, 2022, 2019, 2020, 201...",30,0,0.0,2
6,supported_platforms,object,"[['windows'], ['windows', 'mac', 'linux'], ['w...",7,0,0.0,2
7,price,float64,"[0.0, 4.99, 0.99, 9.99, 1.99, 2.99, 3.99, 14.9...",295,0,0.0,2
8,copiesSold,int64,"[1, 15, 30, 36, 45, 18, 75, 72, 60, 90]",17772,0,0.0,2
9,publisherClass,object,"[Hobbyist, Indie, AA, AAA]",4,0,0.0,2


# Feature Engineering

## 🎮 1. GameRating

We engineered the `GameRating` feature by combining multiple signals that correlate with game sales. Each sub-feature is designed to reflect real-world factors affecting game performance.

---

### 🧩 Components:

#### ✅ `extras_mean`
- Mean of game-related extras: **Achievements**, **Trading Cards**, and **Workshop Support**.
- A small constant (+1) is added to avoid zero values.
- 🧠 **Intuition**: More extras typically lead to higher player engagement → **More Sales** *(Direct Relation)*.

#### ✅ `reviewScore`
- Represents the overall review score of the game.
- Also incremented by +1 to prevent zero values.
- 🧠 **Intuition**: Better reviews attract more players → **More Sales** *(Direct Relation)*.

#### ✅ `publisher_encode`
- Numerical encoding of publisher type:
  - AAA > AA > Indie > Hobbyist
- 🧠 **Intuition**: Well-known publishers usually have more marketing power → **More Sales**.

#### ✅ `release_encode`
- Encoded release date.
- 🧠 **Intuition**: Older games have had more time to accumulate sales → **Inverse Relation**  
  *(i.e., earlier release = more time for copies sold)*.


In [2002]:
publisher_encode = df["publisherClass"].map({ "Hobbyist": 0.01, "Indie": 0.5, "AA": 3, "AAA": 10 })
extras_mean=(df["steam_achievements"]+df["steam_trading_cards"]+df["workshop_support"])/3

# to reduce value of unreleased games
release_encode=df["release_date"].apply(lambda x: '10000' if x in ["Coming soon", "To be announced"] else x)
release_encode=release_encode.astype(int)

df["1-GameRating"]=((extras_mean+1)*(df["reviewScore"]+1)*publisher_encode)/release_encode

In [2003]:
list1=["1-GameRating","copiesSold"]
correlation=df[list1].corr()
correlation

,1-GameRating,copiesSold
1-GameRating,1.000000,0.199933
copiesSold,0.199933,1.000000


## 📊 2. GameRatingWithGenres

- Included the `genres` column in our `GameRating` feature.  
- Improved correlation from **0.127** → **0.151**.

### 🧩 Steps:
1. Get total `copiesSold` for each unique genre across the dataframe.
2. Replace every row in `genres` with the **mean** of `copiesSold` of its genres.
3. Divide by **10 million** to make the values smaller.
4. Multiply `GameRating` by the new `genres` value to create the new feature.


In [2004]:
df['genresTemp'] = df['genres'].apply(lambda x: [genre.strip() for genre in x.split(',')])
from collections import Counter

genre_counter = Counter([genre for sublist in df['genresTemp'] for genre in sublist])
common_genres = {genre for genre, count in genre_counter.items()}

dict1={}
for i in common_genres:
    dict1[i]=0
for idx,genre in df["genresTemp"].items():
    for j in genre:
        dict1[j] += df.loc[idx, "copiesSold"]

genre_means = []

for idx, genre_list in df["genresTemp"].items():
    mean1 = 0
    for genre in genre_list:
        mean1 += dict1[genre]
    mean1 /= len(genre_list)
    genre_means.append(mean1)

df.drop("genresTemp",inplace=True,axis=1)
df["2-GameRatingWithGenres"] = (genre_means * df["1-GameRating"])/1e7

In [2005]:
li=["2-GameRatingWithGenres","copiesSold"]
cor=df[li].corr()
cor

,2-GameRatingWithGenres,copiesSold
2-GameRatingWithGenres,1.000000,0.197091
copiesSold,0.197091,1.000000


# 3- RatingOverPrice

In [2006]:
df["3-RatingOverPrice"]=df["2-GameRatingWithGenres"]/(df["price"]+1)

In [2007]:
li=["3-RatingOverPrice","copiesSold"]
cor=df[li].corr()
cor

,3-RatingOverPrice,copiesSold
3-RatingOverPrice,1.000000,0.334771
copiesSold,0.334771,1.000000


In [2008]:
d=df["copiesSold"].groupby(df["supported_platforms"]).mean()
d.sort_values(ascending=False)

supported_platforms
['windows', 'linux']           316682.495663
['windows', 'mac']             122515.366342
['windows', 'mac', 'linux']    120414.702320
['windows']                     62426.017278
['mac', 'linux']                 3827.000000
['mac']                           497.500000
['linux']                         360.000000
Name: copiesSold, dtype: float64

# 4- GameRatingWithPlatforms

In [ ]:
import ast

li=[]
dict2={"windows":10,"mac":0.01,"linux":30}
for idx,platform in df["supported_platforms"].items():
    sum1=0
    platform_list = ast.literal_eval(platform)
    for j in platform_list:
        sum1+=dict2[j]

    li.append(sum1)

df["4-GameRatingWithPlatforms"]=df["3-RatingOverPrice"]*li

In [2010]:
li=["4-GameRatingWithPlatforms","copiesSold"]
cor=df[li].corr()
cor

,4-GameRatingWithPlatforms,copiesSold
4-GameRatingWithPlatforms,1.000000,0.381038
copiesSold,0.381038,1.000000


In [2011]:
data_info(df)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,name_x,object,"[Delirium, Echoes, Arena, Dodge, Zombie Hunter...",67529,0,0.0,2
1,steam_achievements,bool,"[True, False]",2,0,0.0,2
2,steam_trading_cards,bool,"[False, True]",2,0,0.0,2
3,workshop_support,bool,"[False, True]",2,0,0.0,2
4,genres,object,"[Casual, Indie, Action, Indie, Action, Adventu...",2016,0,0.0,2
5,release_date,object,"[2024, 2021, 2023, 2018, 2022, 2019, 2020, 201...",30,0,0.0,2
6,supported_platforms,object,"[['windows'], ['windows', 'mac', 'linux'], ['w...",7,0,0.0,2
7,price,float64,"[0.0, 4.99, 0.99, 9.99, 1.99, 2.99, 3.99, 14.9...",295,0,0.0,2
8,copiesSold,int64,"[1, 15, 30, 36, 45, 18, 75, 72, 60, 90]",17772,0,0.0,2
9,publisherClass,object,"[Hobbyist, Indie, AA, AAA]",4,0,0.0,2


In [2012]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_and_lemmatize(text):
    words = text.split()
    cleaned_words = []
    for word in words:
        word = word.strip().lower()
        word = re.sub(r'[^a-zA-Z]', '', word)
        if word and word not in stop_words:
            lemma = lemmatizer.lemmatize(word)
            cleaned_words.append(lemma)
    return cleaned_words

df['name'] = df['name_x'].apply(clean_and_lemmatize)

dict12={}
for idx,name in df["name"].items():
    for j in name:
        if j not in dict12:
            dict12[j]=0
        dict12[j] += df.loc[idx, "copiesSold"]

name_means = []

for idx, names_list in df["name"].items():
    mean1 = 0
    if len(names_list) != 0:
        for name in names_list:
            mean1 += dict12[name]
        mean1 /= len(names_list)
    name_means.append(mean1)

df.drop(["name_x","name"],inplace=True,axis=1)
df["5-NameAsCopiesSold"] = name_means
df["5-NameAsCopiesSold"] /= 1e4

In [2013]:
df["6-GameRatingWithNames"]=df["5-NameAsCopiesSold"]*df["4-GameRatingWithPlatforms"]

In [2014]:
li=["1-GameRating","2-GameRatingWithGenres","3-RatingOverPrice","4-GameRatingWithPlatforms","6-GameRatingWithNames","copiesSold"]
cor=df[li].corr()
cor

,1-GameRating,2-GameRatingWithGenres,3-RatingOverPrice,4-GameRatingWithPlatforms,6-GameRatingWithNames,copiesSold
1-GameRating,1.000000,0.916344,0.423460,0.423057,0.147020,0.199933
2-GameRatingWithGenres,0.916344,1.000000,0.428999,0.428111,0.154571,0.197091
3-RatingOverPrice,0.423460,0.428999,1.000000,0.993010,0.425764,0.334771
4-GameRatingWithPlatforms,0.423057,0.428111,0.993010,1.000000,0.477388,0.381038
6-GameRatingWithNames,0.147020,0.154571,0.425764,0.477388,1.000000,0.848744
copiesSold,0.199933,0.197091,0.334771,0.381038,0.848744,1.000000


In [2015]:
df.drop_duplicates(inplace=True)

# Splitting The Data

In [2016]:
X=df.drop("copiesSold",axis=1)
y=df["copiesSold"]

In [2017]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2018]:
y_train = np.log1p(y_train)
y_test = np.log1p(y_test)

In [2019]:
data_info(df)

,Name,Data_Type,Top_10_Unique_Values,Nunique_Values,Nulls,Percent_of_Nulls,Duplicates
0,steam_achievements,bool,"[True, False]",2,0,0.0,0
1,steam_trading_cards,bool,"[False, True]",2,0,0.0,0
2,workshop_support,bool,"[False, True]",2,0,0.0,0
3,genres,object,"[Casual, Indie, Action, Indie, Action, Adventu...",2016,0,0.0,0
4,release_date,object,"[2024, 2021, 2023, 2018, 2022, 2019, 2020, 201...",30,0,0.0,0
5,supported_platforms,object,"[['windows'], ['windows', 'mac', 'linux'], ['w...",7,0,0.0,0
6,price,float64,"[0.0, 4.99, 0.99, 9.99, 1.99, 2.99, 3.99, 14.9...",295,0,0.0,0
7,copiesSold,int64,"[1, 15, 30, 36, 45, 18, 75, 72, 60, 90]",17772,0,0.0,0
8,publisherClass,object,"[Hobbyist, Indie, AA, AAA]",4,0,0.0,0
9,reviewScore,int64,"[100, 0, 50, 67, 75, 80, 83, 88, 86, 89]",99,0,0.0,0


# Scaling

In [2020]:
numerical_columns=["price","1-GameRating","2-GameRatingWithGenres","4-GameRatingWithPlatforms",
                   "3-RatingOverPrice","5-NameAsCopiesSold","6-GameRatingWithNames"]

Ro_scaler = RobustScaler()
x_train[numerical_columns] = Ro_scaler.fit_transform(x_train[numerical_columns])
x_test[numerical_columns] = Ro_scaler.transform(x_test[numerical_columns])

minmax_scaler = MinMaxScaler()
x_train["reviewScore"] = minmax_scaler.fit_transform(x_train[["reviewScore"]])
x_test["reviewScore"] = minmax_scaler.transform(x_test[["reviewScore"]])

# Encoding

one_hot with binarizer

In [2021]:
x_train['genres'] = x_train['genres'].apply(lambda x: [g.strip() for g in x.split(',')])
x_test['genres'] = x_test['genres'].apply(lambda x: [g.strip() for g in x.split(',')])

mlb_genres = MultiLabelBinarizer()
genres_train_encoded = pd.DataFrame(mlb_genres.fit_transform(x_train['genres']), columns=mlb_genres.classes_, index=x_train.index)
genres_test_encoded = pd.DataFrame(mlb_genres.transform(x_test['genres']), columns=mlb_genres.classes_, index=x_test.index)

x_train = x_train.drop(columns=['genres'])
x_test = x_test.drop(columns=['genres'])

x_train = pd.concat([x_train, genres_train_encoded], axis=1)
x_test = pd.concat([x_test, genres_test_encoded], axis=1)

In [2022]:
x_train['supported_platforms'] = x_train['supported_platforms'].apply(eval)
x_test['supported_platforms'] = x_test['supported_platforms'].apply(eval)

mlb_genres = MultiLabelBinarizer()
genres_train_encoded = pd.DataFrame(mlb_genres.fit_transform(x_train['supported_platforms']), columns=mlb_genres.classes_, index=x_train.index)
genres_test_encoded = pd.DataFrame(mlb_genres.transform(x_test['supported_platforms']), columns=mlb_genres.classes_, index=x_test.index)

x_train = x_train.drop(columns=['supported_platforms'])
x_test = x_test.drop(columns=['supported_platforms'])

x_train = pd.concat([x_train, genres_train_encoded], axis=1)
x_test = pd.concat([x_test, genres_test_encoded], axis=1)

In [2023]:
x_train["release_date"] = x_train["release_date"].astype(int)
x_test["release_date"] = x_test["release_date"].astype(int)

oneHot for remaining 

In [2024]:
one_Hot_list = ["steam_achievements", "steam_trading_cards", "workshop_support","publisherClass"]
OneHot_Encoder = OneHotEncoder(handle_unknown='ignore') 

encoded_train = OneHot_Encoder.fit_transform(x_train[one_Hot_list])
encoded_test = OneHot_Encoder.transform(x_test[one_Hot_list])

new_columns = OneHot_Encoder.get_feature_names_out(one_Hot_list)

x_train = pd.concat([
    x_train.drop(columns=one_Hot_list), 
    pd.DataFrame(encoded_train, columns=new_columns, index=x_train.index)  
], axis=1)

x_test = pd.concat([
    x_test.drop(columns=one_Hot_list),  
    pd.DataFrame(encoded_test, columns=new_columns, index=x_test.index)  
], axis=1)

x_train.head()

,release_date,price,reviewScore,1-GameRating,2-GameRatingWithGenres,3-RatingOverPrice,4-GameRatingWithPlatforms,5-NameAsCopiesSold,6-GameRatingWithNames,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,Free To Play,Game Development,Gore,Indie,Massively Multiplayer,Nudity,RPG,Racing,Sexual Content,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Violent,Web Publishing,linux,mac,windows,steam_achievements,steam_trading_cards,workshop_support,publisherClass_1,publisherClass_2,publisherClass_3,publisherClass_4
54352,2020,0.000000,0.00,-0.027752,-0.033593,-0.174081,-0.170506,-0.226444,-0.079213,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False,False,1,0,0,0
152,2022,-0.444444,0.96,1.372642,1.711000,8.512557,11.000196,-0.256071,0.090113,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,True,True,True,0,1,0,0
45187,2019,0.555556,0.57,0.810725,1.106676,0.853673,0.773519,-0.186469,0.136781,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,True,False,False,0,1,0,0
38784,2022,-0.444444,1.00,0.001127,0.012102,0.054316,0.039284,-0.261007,-0.079253,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False,False,1,0,0,0
57856,2021,2.777778,0.80,1.142180,1.688618,0.376151,0.334899,0.879991,1.691787,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False,False,0,1,0,0


# correlation

binary

In [2025]:
cols = ["steam_achievements", "steam_trading_cards", "workshop_support"]

for col in cols:
    corr, p = pointbiserialr(x_train[col], y_train)
    print(f"{col} → Correlation: {corr:.3f}, p-value: {p:.3f}")

steam_achievements → Correlation: 0.172, p-value: 0.000
steam_trading_cards → Correlation: 0.177, p-value: 0.000
workshop_support → Correlation: -0.072, p-value: 0.000


categorical (Anova)

In [2026]:
li=["price","reviewScore","1-GameRating","2-GameRatingWithGenres","3-RatingOverPrice",
"4-GameRatingWithPlatforms","5-NameAsCopiesSold","6-GameRatingWithNames",
"steam_achievements","steam_trading_cards","workshop_support"]

te=x_train.drop(columns=li,axis=1)
f_scores, p_values = f_regression(te, y_train)

correlation_df = pd.DataFrame({
    'Feature': te.columns,
    'F-Score': f_scores,
    'p-value': p_values
}).sort_values(by='F-Score', ascending=False)

print(correlation_df)


                  Feature       F-Score        p-value
30       publisherClass_1  25310.877262   0.000000e+00
31       publisherClass_2  12107.357173   0.000000e+00
0            release_date   9234.849637   0.000000e+00
33       publisherClass_4   4519.019122   0.000000e+00
32       publisherClass_3   2780.135466   0.000000e+00
10           Free To Play   2406.972601   0.000000e+00
6                  Casual    910.672657  1.914705e-198
19             Simulation    495.394868  2.882365e-109
14  Massively Multiplayer    436.957299   1.175955e-96
16                    RPG    435.633103   2.271651e-96
13                  Indie    209.048835   2.701374e-47
28                    mac    170.427519   6.811349e-39
8            Early Access    134.857336   3.855068e-31
22               Strategy     85.542199   2.345914e-20
3               Adventure     77.265630   1.536251e-18
17                 Racing     17.507208   2.866607e-05
15                 Nudity     12.198890   4.785519e-04
27        

In [2027]:
drop=[]
for idx, row in correlation_df.iterrows():
    if row["F-Score"] < 10:
        drop.append(row["Feature"])

x_train.drop(columns=drop,inplace=True)
x_test.drop(columns=drop,inplace=True)

numerical correlation

In [2028]:
li=["price","reviewScore","1-GameRating","2-GameRatingWithGenres","3-RatingOverPrice",
"4-GameRatingWithPlatforms","5-NameAsCopiesSold","6-GameRatingWithNames"]

train_data = pd.concat([x_train, y_train], axis=1)

cor=train_data[li].corr()
cor

,price,reviewScore,1-GameRating,2-GameRatingWithGenres,3-RatingOverPrice,4-GameRatingWithPlatforms,5-NameAsCopiesSold,6-GameRatingWithNames
price,1.000000,-0.016213,0.192792,0.167685,-0.029304,-0.030010,0.036943,-0.005965
reviewScore,-0.016213,1.000000,0.108662,0.092776,0.051729,0.054062,-0.026459,0.009387
1-GameRating,0.192792,0.108662,1.000000,0.915897,0.422579,0.424293,0.119565,0.138103
2-GameRatingWithGenres,0.167685,0.092776,0.915897,1.000000,0.426526,0.427965,0.109612,0.147670
3-RatingOverPrice,-0.029304,0.051729,0.422579,0.426526,1.000000,0.993488,0.081187,0.403653
4-GameRatingWithPlatforms,-0.030010,0.054062,0.424293,0.427965,0.993488,1.000000,0.086317,0.442890
5-NameAsCopiesSold,0.036943,-0.026459,0.119565,0.109612,0.081187,0.086317,1.000000,0.169188
6-GameRatingWithNames,-0.005965,0.009387,0.138103,0.147670,0.403653,0.442890,0.169188,1.000000


In [2030]:
x_train.drop(columns=["4-GameRatingWithPlatforms","2-GameRatingWithGenres"],inplace=True)
x_test.drop(columns=["4-GameRatingWithPlatforms","2-GameRatingWithGenres"],inplace=True)

In [ ]:
x_train.shape

(55433, 26)

# model training

# Linear regresion

In [2032]:
lr=LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2= r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Squared Error: 3.759490651962817
R^2 Score: 0.6176127857974227


# Ridge Regression

In [2033]:
# model = Ridge(
#     alpha=1,
#     fit_intercept=True,
#     solver='auto',
#     random_state=42
# )

# model.fit(x_train, y_train)
# y_pred = model.predict(x_test)

# mse = mean_squared_error(y_test, y_pred)
# r2= r2_score(y_test, y_pred)

# print("Mean Squared Error:", mse)
# print("R^2 Score:", r2)

# Lasso Regression

In [2034]:
# model = Lasso(
#     alpha=0.0001,
#     fit_intercept=True,
#     max_iter=1000,
#     tol=0.0001,
#     random_state=42
# )

# model.fit(x_train, y_train)
# y_pred = model.predict(x_test)

# mse = mean_squared_error(y_test, y_pred)
# r2= r2_score(y_test, y_pred)

# print("Mean Squared Error:", mse)
# print("R^2 Score:", r2)

# RandomForest & GridSearch

In [2035]:
# rf = RandomForestRegressor(random_state=42)

# param_grid = {
#     'n_estimators': [50, 100, 200],  
#     'max_depth': [None, 10, 20, 30],  
#     'min_samples_split': [2, 5, 10],  
#     'min_samples_leaf': [1, 2, 4],    
#     'max_features': ['auto', 'sqrt', 'log2']  
# }

# grid_search = GridSearchCV(
#     estimator=rf,
#     param_grid=param_grid,
#     cv=5,
#     scoring='neg_mean_squared_error',  
#     n_jobs=-1,  
#     verbose=1
# )

# grid_search.fit(x_train, y_train)
# best_rf = grid_search.best_estimator_
# y_pred = best_rf.predict(x_test)

# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print("Best Hyperparameters from GridSearchCV:", grid_search.best_params_)
# print("\nTest Set Evaluation using Best Model from GridSearchCV:")
# print("Mean Squared Error:", mse)
# print("R^2 Score:", r2)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Hyperparameters from GridSearchCV: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}

Test Set Evaluation using Best Model from GridSearchCV:
Mean Squared Error: 2.156030808673064
R^2 Score: 0.7916871819441474

----------------------------------------------------------------------------------------------------------------------

# RandomForest w/ best parameters from GridSearch

In [2036]:
rf=RandomForestRegressor(max_depth= None, max_features='sqrt', min_samples_leaf= 1, min_samples_split=5, n_estimators=200)

rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2= r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Squared Error: 1.997186030600755
R^2 Score: 0.7968612045658362


# SVM & PCA

In [2037]:
# pca = PCA(n_components=None)  
# x_trainP = pca.fit_transform(x_train)
# x_testP = pca.transform(x_test)

# svm = SVR(kernel='rbf', C=10.0, epsilon=0.01)

# svm.fit(x_trainP, y_train)
# y_pred = svm.predict(x_testP)

# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print("Mean Squared Error:", mse)
# print("R^2 Score:", r2)

# XGBOOST & GridSearch

In [2038]:
# xgb = XGBRegressor(
#     objective='reg:squarederror',
#     random_state=42,
#     n_jobs=-1
# )

# param_grid = {
#     'n_estimators': [100, 200, 300],             
#     'learning_rate': [0.01, 0.05, 0.1, 0.2],     
#     'max_depth': [3, 5, 7, 9],                    
#     'subsample': [0.6, 0.8, 1.0],                 
#     'colsample_bytree': [0.6, 0.8, 1.0]          
# }

# grid_search = GridSearchCV(
#     estimator=xgb,
#     param_grid=param_grid,
#     cv=5,
#     scoring='r2',
#     n_jobs=-1,
#     verbose=1
# )

# grid_search.fit(x_train, y_train)

# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(x_test)

# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print("Best Hyperparameters:", grid_search.best_params_)
# print("Mean Squared Error:", mse)
# print("R^2 Score:", r2)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best Hyperparameters: {'colsample_bytree': 0.6, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 300, 'subsample': 1.0}
Mean Squared Error: 2.1214420830530742
R^2 Score: 0.7950290984315539

# XGBOOST w/ Best Parameters from Grid Search

In [2039]:
model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=7,
    subsample=1,
    colsample_bytree=0.7,
    random_state=42,
    n_jobs=-1
)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Squared Error: 1.9157507287261746
R^2 Score: 0.8051441931683769


# ELastic

In [2040]:
# from sklearn.linear_model import ElasticNet

# model = ElasticNet(max_iter=10000, random_state=42)

# param_grid = {
#     'alpha': [0.01, 0.1, 1.0, 10.0],
#     'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
# }

# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
# grid_search.fit(x_train, y_train)

# best_model = grid_search.best_estimator_
# print("Best Params:", grid_search.best_params_)

# y_pred = best_model.predict(x_test)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"Test MSE: {mse:.4f}")
# print(f"Test R² Score: {r2:.4f}")

# LGB

In [2041]:
# import lightgbm as lgb
# model = lgb.LGBMRegressor(random_state=42)

# param_grid = {
#     'n_estimators': [100, 200],
#     'learning_rate': [0.05, 0.1],
#     'num_leaves': [31, 50],
#     'max_depth': [-1, 10]
# }

# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
# grid_search.fit(x_train, y_train)

# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(x_test)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print("Best LightGBM Params:", grid_search.best_params_)
# print(f"Test MSE: {mse:.4f}")
# print(f"Test R² Score: {r2:.4f}")

In [2042]:
# model = lgb.LGBMRegressor(
#     n_estimators=270,
#     learning_rate=0.06,
#     num_leaves=50,
#     max_depth=10,
#     random_state=42,
#     min_child_samples=20,
# )

# model.fit(x_train, y_train)

# y_pred = model.predict(x_test)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"Test MSE: {mse:.4f}")
# print(f"Test R² Score: {r2:.4f}")

# CatBOOSt

In [2043]:
# model = CatBoostRegressor(verbose=0, random_state=42)

# param_grid = {
#     'iterations': [200, 300],
#     'learning_rate': [0.03, 0.1],
#     'depth': [6, 10]
# }

# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
# grid_search.fit(x_train, y_train)

# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(x_test)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print("Best CatBoost Params:", grid_search.best_params_)
# print(f"Test MSE: {mse:.4f}")
# print(f"Test R² Score: {r2:.4f}")

In [2044]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(
    iterations=280,
    learning_rate=0.06,
    depth=11,
    verbose=0,
    random_state=42
)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {mse:.4f}")
print(f"Test R² Score: {r2:.4f}")


Test MSE: 1.9269
Test R² Score: 0.8040
